We note

$u(-x, y) = u(x, y) \Rightarrow \partial_x u |_{x=0} = 0$

$u(x, -y) = -u(x, y) \Rightarrow u(x, 0) = 0$


Hence the new problem

\begin{cases}
-\epsilon_0 \Delta u = 0 \quad in \quad \Omega \\
u = 1  \quad on \quad \Gamma_{el.1}\\
u=0  \quad on  \quad\Gamma_{b} \\
\partial_n u = 0 \quad on \quad \Gamma_{l}\cup\Gamma_{t}\cup\Gamma_{r}
\end{cases}


$\Omega = (0,5) \times (0,5) - \Omega_{el.1}$ 


$\Omega_{el.1} = (0, 0.4) \times (0.3, 0.4)$

In [1]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *
from netgen.csg import *
import numpy as np
import matplotlib.pyplot as plt
e0 =  8.854e-12

In [43]:
outer = Rectangle(5, 5).Face()
outer.edges.Max(X).name = "r"
outer.edges.Min(X).name = "l"
outer.edges.Min(Y).name = "b"
outer.edges.Max(Y).name = "t"


electrode = MoveTo(0,0.3).Rectangle(0.4, 0.1).Face()
electrode.edges.name="inner"
electrode.col = (1,0,0)


geo = Glue([outer, electrode])


In [48]:
mesh = Mesh(OCCGeometry(geo, dim=2).GenerateMesh(maxh=0.05))
fes = H1(mesh, order =2, dirichlet="inner|b")

u, v = fes.TnT()
gfu = GridFunction(fes)

bdry_values = {'inner':1}
gfu.Set(mesh.BoundaryCF(bdry_values), definedon=mesh.Boundaries("inner|b"))

a = BilinearForm(fes)
a += e0*grad(u)*grad(v)*dx
a.Assemble()

r = - a.mat * gfu.vec
gfu.vec.data += a.mat.Inverse(freedofs=fes.FreeDofs()) * r
E = -grad(gfu)

In [49]:
Draw(gfu, mesh, vectors=True)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [50]:
def calc_energy(E, mesh):
    energy_density = 0.5*e0*InnerProduct(E,E)
    energy = Integrate(energy_density, mesh)
    return energy

en = calc_energy(E, mesh)

From the previous exercise we found that the energy is approximately 4.8e-11 J (since we are in S.I)

In [51]:
4*en

4.78189884092603e-11

Same here